<a href="https://colab.research.google.com/github/shreyac435/math-question-classifier/blob/main/math_question_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import zipfile
import json

os.listdir('/content')
zipname = 'dataset.zip'
zipfile_obj = zipfile.ZipFile('/content/' + zipname, 'r') #unzipped the file here
zipfile_obj.extractall('/content/data')
zipfile_obj.close()
os.listdir('/content/data')
os.listdir('/content/data/MATH')
os.listdir('/content/data/MATH/train')



['algebra',
 'prealgebra',
 'number_theory',
 'counting_and_probability',
 'geometry',
 'precalculus',
 'intermediate_algebra']

In [ ]:
questions = []
topics = []
basepath = '/content/data/MATH/train'

for folder in os.listdir(basepath):
    folderpath = basepath + '/' + folder
    for file in os.listdir(folderpath):
        filepath = folderpath + '/' + file
        f = open(filepath, 'r')
        data = json.load(f)
        f.close()
        questions.append(data['problem'])
        topics.append(data['type'])
print(len(questions))
print(questions[0])
print(topics[0])


7500
What is the minimum value of $z$ if $z=x^2+2y^2+6x-4y+22?$
Algebra


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
qtrain, qtest, ttrain, ttest = train_test_split(
    questions, topics, test_size=0.2, random_state=20)
vectorizer = TfidfVectorizer(
    lowercase=True,
    stop_words='english',
    ngram_range=(1, 2))
qtrain_vec = vectorizer.fit_transform(qtrain)
qtest_vec = vectorizer.transform(qtest)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
nb_model = MultinomialNB()
nb_model.fit(qtrain_vec, ttrain)
nb_pred = nb_model.predict(qtest_vec)
nb_acc = accuracy_score(ttest, nb_pred)
print("Naive Bayes Accuracy:", nb_acc)

Naive Bayes Accuracy: 0.6786666666666666


In [ ]:
from sklearn.svm import LinearSVC
svm_model = LinearSVC()
svm_model.fit(qtrain_vec, ttrain)
svm_pred = svm_model.predict(qtest_vec)
svm_acc = accuracy_score(ttest, svm_pred)
print("Linear SVM Accuracy:", svm_acc)

Linear SVM Accuracy: 0.734


In [ ]:
for i in range(5):
    print("ACTUAL TOPIC:")
    print(ttest[i])
    print("NB PREDICTED:")
    print(nb_pred[i])
    print("SVM PREDICTED:")
    print(svm_pred[i])
    print("\n")

ACTUAL TOPIC:
Algebra
NB PREDICTED:
Algebra
SVM PREDICTED:
Algebra


ACTUAL TOPIC:
Counting & Probability
NB PREDICTED:
Number Theory
SVM PREDICTED:
Number Theory


ACTUAL TOPIC:
Precalculus
NB PREDICTED:
Precalculus
SVM PREDICTED:
Precalculus


ACTUAL TOPIC:
Number Theory
NB PREDICTED:
Number Theory
SVM PREDICTED:
Number Theory


ACTUAL TOPIC:
Algebra
NB PREDICTED:
Algebra
SVM PREDICTED:
Algebra




In [ ]:
sample_qs = []
for folder in os.listdir(basepath):
    folderpath = basepath + '/' + folder
    for file in os.listdir(folderpath):
        filepath = folderpath + '/' + file
        f = open(filepath, 'r')
        data = json.load(f)
        f.close()
        sample_qs.append(data['problem'])
        if len(sample_qs) == 10:
            break
    if len(sample_qs) == 10:
        break
sample_qs[:3]

['What is the minimum value of $z$ if $z=x^2+2y^2+6x-4y+22?$',
 'If $a*b=a^2+ab-b^2$, find $3*2$.',
 'If a 3\'\' by 3\'\' square is added at each successive stage, what will be the area of the rectangle at Stage 6, in square inches?\n\n[asy]size(250); real textsize = 10pt;\ndraw(unitsquare);\ndraw(shift(1.5*right)*unitsquare);\ndraw(shift(2.5*right)*unitsquare);\ndraw(shift(4*right)*unitsquare);\ndraw(shift(5*right)*unitsquare);\ndraw(shift(6*right)*unitsquare);\nlabel("Stage 1",(.5,0),S,fontsize(textsize));\nlabel("Stage 2",(2.5,0),S,fontsize(textsize));\nlabel("Stage 3",(5.5,0),S,fontsize(textsize));[/asy]']

In [ ]:
!pip install -q transformers accelerate sentencepiece

In [ ]:

import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-xl"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)

def explain_question(q):
    prompt = "Explain step by step in detail for a high school student:\n" + q
    tokens = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(**tokens, max_new_tokens=200)
    return tokenizer.decode(output[0], skip_special_tokens=True)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/accelerate/utils/modeling.py:1598: UserWarning: The following device_map keys do not match any submodules in the model: ['decoder.embed_tokens']
  warnings.warn(


In [ ]:
for q in sample_qs[:2]:
    print("QUESTION:")
    print(q)
    print("\nLLM EXPLANATION:")
    print(explain_question(q))

QUESTION:
What is the minimum value of $z$ if $z=x^2+2y^2+6x-4y+22?$

LLM EXPLANATION:
The minimum value of $z$ is 0 because z is the product of x and y. The answer: 0 because z is the product of x and y.
QUESTION:
If $a*b=a^2+ab-b^2$, find $3*2$.

LLM EXPLANATION:


In [ ]:
while True:
    user_q = input("\nEnter a math question (or type exit): ")
    if user_q.lower() == "exit":
        break
    user_vec = vectorizer.transform([user_q])
    predicted_topic = svm_model.predict(user_vec)[0]
    print("Predicted Topic:", predicted_topic)
    print("\nExplanation:\n")
    print(explain_question(user_q))

#just a demo, isn't properly working, some questions we can ask
#solve for x, x+3=0


Enter a math question (or type exit): find integral of x^3
Predicted Topic: Number Theory

Explanation:



KeyboardInterrupt: 

In [ ]:
!pip install -q wandb

import wandb

wandb.login()

wandb.init(project="math-question-classifier")

wandb.log({
    "Naive Bayes Accuracy": nb_acc,
    "Linear SVM Accuracy": svm_acc,
    "Features": "TF-IDF (unigram + bigram)"
})

wandb.finish()
